In [1]:
import tensorflow as tf
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack

from numpy.random import randn
from numpy.random import randint

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout

from matplotlib import pyplot

In [2]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [5]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [6]:
######### to import data from google drive

import cv2
import os
import pandas as pd

path = '/content/drive/MyDrive/Colab Notebooks/Traffic signs/myData'
labelsFile = '/content/drive/MyDrive/Colab Notebooks/Traffic signs/labels.csv'
imageDimensions = (32,32,3)

count = 0 
images = []  #features
classno = []  #labels
Label_list = pd.read_csv(labelsFile)
print("Total Classes detected :",len(Label_list))

num_of_classes = len(Label_list)
print("Importing all classes......")
print(" ")

for count in range(num_of_classes):
    img_list_per_class = os.listdir(path+"/"+str(count))
    for y in img_list_per_class:
        clrImg = cv2.imread(path+"/"+str(count)+"/"+y)
        images.append(clrImg)
        classno.append(count)
    #print(len(images))
    #print(count, end=" ")  #cursor doesnt go to new
    print("Importing class "+str(count))
    print("no of images in class "+str(count)+" are "+str(len(img_list_per_class)))
    count = count+1

print()    
print('total data set'+str(len(images)))




Total Classes detected : 43
Importing all classes......
 
Importing class 0
no of images in class 0 are 180
Importing class 1
no of images in class 1 are 1980
Importing class 2
no of images in class 2 are 2010
Importing class 3
no of images in class 3 are 1260
Importing class 4
no of images in class 4 are 1770
Importing class 5
no of images in class 5 are 1650
Importing class 6
no of images in class 6 are 360
Importing class 7
no of images in class 7 are 1290
Importing class 8
no of images in class 8 are 1260
Importing class 9
no of images in class 9 are 1320
Importing class 10
no of images in class 10 are 1800
Importing class 11
no of images in class 11 are 1170
Importing class 12
no of images in class 12 are 1890
Importing class 13
no of images in class 13 are 1930
Importing class 14
no of images in class 14 are 690
Importing class 15
no of images in class 15 are 540
Importing class 16
no of images in class 16 are 360
Importing class 17
no of images in class 17 are 990
Importing clas

In [7]:
from sklearn.model_selection import train_test_split


batch_size = 64
x_train, x_test, y_train, y_test =train_test_split(images, classno, test_size = 0.1)
print(x_train[0].shape)


(32, 32, 3)


In [8]:
from sklearn.model_selection import train_test_split
# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	#(trainX, _), (_, _) = load_data()
	trainX ,_ ,_ ,_  =train_test_split(images, classno, test_size = 0.01)
  # convert from unsigned ints to floats

	X = np.array(trainX, dtype=np.float32)
	#X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

In [9]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y
 

In [10]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [11]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [12]:
import os
import cv2
import sys
from PIL import Image
DATA_PATH = "D:/projectwork/images_gan_TS_RGB/"

# create and save a plot of generated images
def save_plot(examples, epoch):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(20):
		img = tf.keras.preprocessing.image.array_to_img(examples[i])
		img.save("generated_img_{epoch}_{i}.png".format(epoch=epoch+1,i=i+1))
		#cv2.imwrite(os.path.join(DATA_PATH, "generated_img_{epoch}_{i}.png".format(epoch=epoch, i=i)), img)
  

In [13]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [14]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 5 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [15]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
>32, 120/269, d1=0.580, d2=0.565 g=1.106
>32, 121/269, d1=0.506, d2=0.636 g=1.217
>32, 122/269, d1=0.668, d2=0.440 g=1.207
>32, 123/269, d1=0.629, d2=0.531 g=1.153
>32, 124/269, d1=0.524, d2=0.494 g=1.260
>32, 125/269, d1=0.662, d2=0.505 g=1.154
>32, 126/269, d1=0.569, d2=0.565 g=1.211
>32, 127/269, d1=0.582, d2=0.569 g=1.138
>32, 128/269, d1=0.548, d2=0.550 g=1.095
>32, 129/269, d1=0.491, d2=0.495 g=1.163
>32, 130/269, d1=0.569, d2=0.569 g=1.139
>32, 131/269, d1=0.632, d2=0.482 g=1.175
>32, 132/269, d1=0.687, d2=0.499 g=1.069
>32, 133/269, d1=0.416, d2=0.578 g=1.157
>32, 134/269, d1=0.565, d2=0.508 g=1.157
>32, 135/269, d1=0.588, d2=0.468 g=1.149
>32, 136/269, d1=0.563, d2=0.591 g=1.134
>32, 137/269, d1=0.538, d2=0.502 g=1.135
>32, 138/269, d1=0.647, d2=0.517 g=1.117
>32, 139/269, d1=0.612, d2=0.492 g=1.013
>32, 140/269, d1=0.564, d2=0.690 g=1.109
>32, 141/269, d1=0.685, d2=0.562 g=1.107
>32, 142/269, d1=0.609, d2=0.547 g=1.247
>32, 1